#  IBM Applied D.S Capstone Project (Battle of Neighborhoods)


## Determining the best place to open a new business in outter side of London city limits.


London is definitely one of the best places to start a business. Even though it has high competition it still offers plenty of opportunities if you have a lucrative idea. Being the capital of both England and the UK, it is the center for commerce, finance, fashion, healthcare, research, development, business, tourism, and plenty more. As such there are many opportunities for people in London that want to start a business. But It is also an Expensive place to Live and thrive. Many businesses want to open a venue here and many pay top price to have their window on Piccadilly or Oxford Street. But what about those, 
        
    1. Who want to start their own business and cannot really afford to open in the City yet? 
    2. Where is it best to open a new place? 
    3. Where will it be cheapest and will have enough people living around to be popular? 
    4. Where the competition is not too overwhelming?

Lets find out the answers for the above questions:



## Content in Notebook

<font color=blue> 1. Import Libraries </font>

<font color=blue> 2. Scrap data from webpage into a DataFrame </font>

    
<font color=blue> 3. Exploratory Visualization 1  </font>

    
<font color=blue> 4. Define Foursquare Credentials and Version </font>

<font color=blue> 5. Analysis Using Clustering (K-Means Clustering) </font>

<font color=blue> 6. Exploratory Visualization 2 </font>


<font color=blue> 7. Examine Clusters </font>   

    
<font color=blue> 8. Observations </font>



<font color=blue> 9. Discussion </font>

<font color=blue> 10. Conclusion </font>


<font color=Red> Target Audience</font>

1. First time entrepreneurs, who want to start their first business. Below dataset will give a comprehensive insight into where best to open a new venue, to maximize the value for money.

2. People who already run a business and want to start a new branch. Given the extra information, it may provide some valuable information before decision     making.

# 1. Import Libraries

In [33]:
from geopy.geocoders import Nominatim
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import requests
from sklearn.cluster import KMeans
import matplotlib.cm as cm
import matplotlib.colors as colors

print("Libraries imported.")


Libraries imported.


## 2. Scrap data from webpage into a DataFrame

Name of the London Boroughs, area and population from web scrapping. Filter the boroughs to get the best candidates for the analysis.

In [34]:
## Get the names of Boroughs, Areas, Population, Coordinates from Wikipedia.   
url='https://en.wikipedia.org/wiki/List_of_London_boroughs'

df=pd.read_html(url, header=0)[0]

df.head()

,Borough,Inner,Status,Local authority,Political control,Headquarters,Area (sq mi),Population (2019 est)[1],Co-ordinates,Nr. in map
0,Barking and Dagenham [note 1],NaN,NaN,Barking and Dagenham London Borough Council,Labour,"Town Hall, 1 Town Square",13.93,212906,".mw-parser-output .geo-default,.mw-parser-outp...",25
1,Barnet,NaN,NaN,Barnet London Borough Council,Conservative,"Barnet House, 2 Bristol Avenue, Colindale",33.49,395896,51°37′31″N 0°09′06″W﻿ / ﻿51.6252°N 0.1517°W,31
2,Bexley,NaN,NaN,Bexley London Borough Council,Conservative,"Civic Offices, 2 Watling Street",23.38,248287,51°27′18″N 0°09′02″E﻿ / ﻿51.4549°N 0.1505°E,23
3,Brent,NaN,NaN,Brent London Borough Council,Labour,"Brent Civic Centre, Engineers Way",16.70,329771,51°33′32″N 0°16′54″W﻿ / ﻿51.5588°N 0.2817°W,12
4,Bromley,NaN,NaN,Bromley London Borough Council,Conservative,"Civic Centre, Stockwell Close",57.97,332336,51°24′14″N 0°01′11″E﻿ / ﻿51.4039°N 0.0198°E,20


Processing the Information From Wiki To Make Necessary Lists. Check and Compare with Google Search and Refine if Necessary

In [35]:
# we get the nessessary information on London Boroughs, dropping the extras, that will not be needed for the analysis.

clean_df = df.drop(['Status','Local authority','Political control','Headquarters','Nr. in map'], axis=1)
clean_df['Inner'].replace(np.nan,'0', inplace=True)
clean_df['Borough'].replace('Barking and Dagenham [note 1]','Barking and Dagenham', inplace=True)
clean_df['Borough'].replace('Greenwich [note 2]','Greenwich', inplace=True)
clean_df['Borough'].replace('Hammersmith and Fulham [note 4]','Hammersmith and Fulham', inplace=True)
Inn = ['Camden','Greenwich','Hackney','Hammersmith and Fulham','Islington','Kensington and Chelsea','Lewisham','Lambeth','Southwark','Tower Hamlets','Wandsworth','Westminster']
clean_df.head()
clean_df['Inner'] = '0'
clean_df.head()

,Borough,Inner,Area (sq mi),Population (2019 est)[1],Co-ordinates
0,Barking and Dagenham,0,13.93,212906,".mw-parser-output .geo-default,.mw-parser-outp..."
1,Barnet,0,33.49,395896,51°37′31″N 0°09′06″W﻿ / ﻿51.6252°N 0.1517°W
2,Bexley,0,23.38,248287,51°27′18″N 0°09′02″E﻿ / ﻿51.4549°N 0.1505°E
3,Brent,0,16.70,329771,51°33′32″N 0°16′54″W﻿ / ﻿51.5588°N 0.2817°W
4,Bromley,0,57.97,332336,51°24′14″N 0°01′11″E﻿ / ﻿51.4039°N 0.0198°E


Then we rename the columns, making the dataset better on the eyes. Because of extra notes in the Wiki page, we will rename some of the Boroughs. Due to the staggering difference in rent price, as well as the ammount of venues in London, we will filter to have only the Outer boroughs going forward

In [4]:
clean_df['Inner'] = clean_df.Borough.isin(Inn).astype(int)
Out = clean_df[clean_df.Inner == 0]
Out = Out.drop(['Inner'], axis=1)
ldf = Out.rename(columns = {"Area (sq mi)": "Area", 
                            "Population (2019 est)[1]":"Population"})
geolocator = Nominatim(user_agent="London_explorer")
ldf['Co-ordinates']= ldf['Borough'].apply(geolocator.geocode).apply(lambda x: (x.latitude, x.longitude))
ldf[['Latitude', 'Longitude']] = ldf['Co-ordinates'].apply(pd.Series)
ldf.head()

,Borough,Area,Population,Co-ordinates,Latitude,Longitude
0,Barking and Dagenham,13.93,212906,"(51.5541171, 0.15050434261994267)",51.554117,0.150504
1,Barnet,33.49,395896,"(51.65309, -0.2002261)",51.653090,-0.200226
2,Bexley,23.38,248287,"(39.9692378, -82.936864)",39.969238,-82.936864
3,Brent,16.70,329771,"(32.9373463, -87.1647184)",32.937346,-87.164718
4,Bromley,57.97,332336,"(51.4028046, 0.0148142)",51.402805,0.014814


Finally, we edit the coordinates, that are incorrect. Lets filter the set even more - because rent price is a big issue for the new business, we will get the boroughs with the lowest ones.

In [5]:
Max_Rent = ['102.25','150.75','97','150.75','118.5','129.25','140','102.25','107.75','140','86','161.5','161.5','140','123.75','134.5','118.5','140','129.25','145.25']
ldf['Max_Rent'] = Max_Rent

ldf["Max_Rent"] = pd.to_numeric(ldf["Max_Rent"])
fin = ldf[ldf.Max_Rent <= 125]
fin

Long_list = fin['Longitude'].tolist()
Lat_list = fin['Latitude'].tolist()
print ("Old latitude list: ", Lat_list)
print ("Old Longitude list: ", Long_list)

replace_longitudes = {-106.6621329:0.0799, -2.8417544: 0.1837}
replace_latitudes = {50.7164496:51.6636, 51.0358628: 51.5499}

longtitudes_new = [replace_longitudes.get(n7,n7) for n7 in Long_list]
latitudes_new = [replace_latitudes.get(n7,n7) for n7 in Lat_list]

fin = fin.drop(['Co-ordinates', 'Longitude'], axis=1)

fin['Longitude'] = longtitudes_new
fin['Latitude'] = latitudes_new
fin

Old latitude list:  [51.5541171, 39.9692378, 51.4028046, 51.6520851, 51.587929849999995, 51.5443851, 51.41086985, 51.5763203]
Old Longitude list:  [0.15050434261994267, -82.936864, 0.0148142, -0.0810175, -0.10541010599099046, -0.14430716398919305, -0.18809708858824303, 0.0454097]


,Borough,Area,Population,Latitude,Max_Rent,Longitude
0,Barking and Dagenham,13.93,212906,51.554117,102.25,0.150504
2,Bexley,23.38,248287,39.969238,97.00,-82.936864
4,Bromley,57.97,332336,51.402805,118.50,0.014814
8,Enfield,31.74,333794,51.652085,102.25,-0.081018
12,Haringey,11.42,268647,51.587930,107.75,-0.105410
14,Havering,43.35,259552,51.544385,86.00,-0.144307
22,Merton,14.52,206548,51.410870,123.75,-0.188097
24,Redbridge,21.78,305222,51.576320,118.50,0.045410


Now lets make a map, with all boroughs in question marked up

## 3. Exploratory Visualization 1

In [6]:
address = 'London'

geolocator = Nominatim(user_agent="London_explorer")
location = geolocator.geocode(address)
London_latitude = location.latitude
London_longitude = location.longitude
print('The geograpical coordinates of London are {}, {}.'.format(London_latitude, London_longitude))

The geograpical coordinates of London are 51.5073219, -0.1276474.


In [7]:
!pip install folium
import folium

Fin_Brgh = folium.Map(location=[London_latitude, London_longitude], zoom_start=12)


for lat, lng, label in zip(fin['Latitude'], fin['Longitude'], 
                            fin['Borough']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=9,
        popup=label,
        color='Red',
        fill=True,
        fill_color='#Blue',
        fill_opacity=0.7).add_to(Fin_Brgh)
Fin_Brgh

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
     |████████████████████████████████| 94 kB 3.6 MB/s eta 0:00:011


Now we have a map, its time to get the venues from Foursquare. Due to a big borough area, we will take a rather big radius of search.

## 4. Define Foursquare Credentials and Version

In [8]:
# define Foursquare Credentials and Version
CLIENT_ID = 'XOSLQ4ZIZPWRG4PL1XF1P0NJFDEINY0GZJ2JSCJHNOJUDENY' # your Foursquare ID
CLIENT_SECRET = 'YS2UJSWYRJ1JPHIWVUGRDWN4BNVM1I051FIRIRFA20U0INCP' #your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: XOSLQ4ZIZPWRG4PL1XF1P0NJFDEINY0GZJ2JSCJHNOJUDENY
CLIENT_SECRET:YS2UJSWYRJ1JPHIWVUGRDWN4BNVM1I051FIRIRFA20U0INCP


In [9]:
radius = 5000
LIMIT = 100

def getVenues(names, latitudes, longitudes, radius=5000):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Borough', 
                  'Latitude', 
                  'Longitude', 
                  'Venue', 
                  'Venue_Lat', 
                  'Venue_Long', 
                  'Venue_Category']
    
    return(nearby_venues)

In [10]:
Brgh_Venues = getVenues(names=fin['Borough'],
                        latitudes=fin['Latitude'],
                        longitudes=fin['Longitude'])

Barking and Dagenham
Bexley
Bromley
Enfield
Haringey
Havering
Merton
Redbridge


Lets count the venues in every borough, to get the data scale

In [11]:
Brgh_Venues.groupby('Borough').count()

,Latitude,Longitude,Venue,Venue_Lat,Venue_Long,Venue_Category
Borough,,,,,,
Barking and Dagenham,97,97,97,97,97,97
Bexley,89,89,89,89,89,89
Bromley,100,100,100,100,100,100
Enfield,100,100,100,100,100,100
Haringey,100,100,100,100,100,100
Havering,100,100,100,100,100,100
Merton,100,100,100,100,100,100
Redbridge,100,100,100,100,100,100


Lets break these numbers down to see which venues are more popular than the others

In [12]:
London_Brgh_onehot = pd.get_dummies(Brgh_Venues[['Venue_Category']], prefix="", prefix_sep="")
mid =  Brgh_Venues['Borough']

London_Brgh_onehot.insert(0, 'Borough', mid)

London_Brgh_onehot.head()

,Borough,ATM,American Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Asian Restaurant,Athletics & Sports,Automotive Shop,Bagel Shop,...,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio,Zoo Exhibit
0,Barking and Dagenham,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Barking and Dagenham,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Barking and Dagenham,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Barking and Dagenham,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Barking and Dagenham,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [13]:
Brgh_grouped = London_Brgh_onehot.groupby('Borough').mean().reset_index()
Brgh_grouped

,Borough,ATM,American Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Asian Restaurant,Athletics & Sports,Automotive Shop,Bagel Shop,...,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio,Zoo Exhibit
0,Barking and Dagenham,0.000000,0.00,0.00,0.00,0.000000,0.00,0.00,0.000000,0.000000,...,0.00,0.000000,0.000000,0.00,0.00,0.00,0.000000,0.000000,0.00,0.00
1,Bexley,0.011236,0.00,0.00,0.00,0.011236,0.00,0.00,0.011236,0.011236,...,0.00,0.022472,0.011236,0.00,0.00,0.00,0.011236,0.022472,0.00,0.00
2,Bromley,0.000000,0.01,0.00,0.00,0.000000,0.01,0.01,0.000000,0.000000,...,0.00,0.000000,0.000000,0.00,0.00,0.00,0.000000,0.000000,0.01,0.00
3,Enfield,0.000000,0.00,0.00,0.00,0.000000,0.00,0.01,0.000000,0.000000,...,0.00,0.000000,0.000000,0.00,0.00,0.00,0.000000,0.000000,0.00,0.00
4,Haringey,0.000000,0.00,0.00,0.00,0.000000,0.00,0.00,0.000000,0.000000,...,0.01,0.000000,0.000000,0.00,0.01,0.00,0.000000,0.000000,0.00,0.00
5,Havering,0.000000,0.00,0.01,0.02,0.000000,0.00,0.01,0.000000,0.000000,...,0.00,0.000000,0.000000,0.00,0.00,0.00,0.000000,0.000000,0.01,0.01
6,Merton,0.000000,0.00,0.02,0.00,0.000000,0.02,0.00,0.000000,0.000000,...,0.00,0.000000,0.000000,0.01,0.01,0.00,0.000000,0.000000,0.00,0.00
7,Redbridge,0.000000,0.00,0.00,0.02,0.000000,0.00,0.00,0.000000,0.000000,...,0.02,0.000000,0.000000,0.00,0.01,0.01,0.000000,0.000000,0.00,0.00


With all those zeros, it is easy to assume, that people there just sit at home! Lets see, which of the venues are most popular?

In [14]:
num_top_venues = 5

for brgh in Brgh_grouped['Borough']:
    print("_________"+brgh+"________")
    temp = Brgh_grouped[Brgh_grouped['Borough'] == brgh].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

_________Barking and Dagenham________
           venue  freq
0  Grocery Store  0.10
1    Supermarket  0.09
2           Park  0.09
3            Pub  0.06
4    Coffee Shop  0.06


_________Bexley________
                venue  freq
0         Coffee Shop  0.07
1      Ice Cream Shop  0.04
2         Pizza Place  0.04
3      Discount Store  0.04
4  Chinese Restaurant  0.03


_________Bromley________
           venue  freq
0            Pub  0.10
1    Coffee Shop  0.08
2           Park  0.07
3  Grocery Store  0.07
4    Pizza Place  0.05


_________Enfield________
                venue  freq
0         Coffee Shop  0.08
1                Park  0.07
2                 Pub  0.07
3  Turkish Restaurant  0.07
4                Café  0.06


_________Haringey________
         venue  freq
0          Pub  0.11
1         Café  0.10
2  Coffee Shop  0.09
3         Park  0.07
4       Bakery  0.06


_________Havering________
         venue  freq
0          Pub  0.06
1        Hotel  0.05
2  Pizza Place  0.04
3  C

We all love NumPy Arrays, but the dataframes are better.

In [15]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]


num_top_venues = 5

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Borough']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
brgh_venues_sorted = pd.DataFrame(columns=columns)
brgh_venues_sorted['Borough'] = Brgh_grouped['Borough']

for ind in np.arange(Brgh_grouped.shape[0]):
    brgh_venues_sorted.iloc[ind, 1:] = return_most_common_venues(Brgh_grouped.iloc[ind, :], num_top_venues)

brgh_venues_sorted.head(8)

,Borough,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,Barking and Dagenham,Grocery Store,Park,Supermarket,Pub,Coffee Shop
1,Bexley,Coffee Shop,Ice Cream Shop,Pizza Place,Discount Store,Chinese Restaurant
2,Bromley,Pub,Coffee Shop,Grocery Store,Park,Pizza Place
3,Enfield,Coffee Shop,Park,Pub,Turkish Restaurant,Café
4,Haringey,Pub,Café,Coffee Shop,Park,Turkish Restaurant
5,Havering,Pub,Hotel,Coffee Shop,Park,Bakery
6,Merton,Pub,Park,Bar,Coffee Shop,Sushi Restaurant
7,Redbridge,Pub,Park,Coffee Shop,Restaurant,Café


Because the ammount of boroughs is small, we will have only 3 clusters

## 5. Analysis Using Clustering (K-Means Clustering)

In [16]:
kclusters = 3

brgh_grouped_clustering = Brgh_grouped.drop('Borough', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(brgh_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([2, 0, 2, 2, 1, 0, 1, 1], dtype=int32)

And now - lets join all of our dataframes together for the main event!

In [17]:
brgh_venues_sorted.insert(0, 'Cluster Label', kmeans.labels_)
Borough_merged = pd.merge(fin,brgh_venues_sorted, on='Borough')
Borough_merged

,Borough,Area,Population,Latitude,Max_Rent,Longitude,Cluster Label,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,Barking and Dagenham,13.93,212906,51.554117,102.25,0.150504,2,Grocery Store,Park,Supermarket,Pub,Coffee Shop
1,Bexley,23.38,248287,39.969238,97.00,-82.936864,0,Coffee Shop,Ice Cream Shop,Pizza Place,Discount Store,Chinese Restaurant
2,Bromley,57.97,332336,51.402805,118.50,0.014814,2,Pub,Coffee Shop,Grocery Store,Park,Pizza Place
3,Enfield,31.74,333794,51.652085,102.25,-0.081018,2,Coffee Shop,Park,Pub,Turkish Restaurant,Café
4,Haringey,11.42,268647,51.587930,107.75,-0.105410,1,Pub,Café,Coffee Shop,Park,Turkish Restaurant
5,Havering,43.35,259552,51.544385,86.00,-0.144307,0,Pub,Hotel,Coffee Shop,Park,Bakery
6,Merton,14.52,206548,51.410870,123.75,-0.188097,1,Pub,Park,Bar,Coffee Shop,Sushi Restaurant
7,Redbridge,21.78,305222,51.576320,118.50,0.045410,1,Pub,Park,Coffee Shop,Restaurant,Café


Mapping the clusters. Not what you expect, from our first map

## 6. Exploratory Visualization 2

In [18]:
map_clusters = folium.Map(location=[London_latitude, London_longitude], zoom_start=10)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster, rent, pop in zip(Borough_merged['Latitude'],
                                  Borough_merged['Longitude'],
                                  Borough_merged['Borough'],
                                  Borough_merged['Cluster Label'],
                                  Borough_merged['Max_Rent'],
                                  Borough_merged['Population']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster) + " " + "Rent " + str(rent) + " " + "Population " + str(pop), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=25,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

After seeing the clusters, it is worth looking into each one with a bit more detail, to see, which one is better suited for opening a venue.

## 7. Examine Clusters

In [22]:
Borough_merged.loc[Borough_merged['Cluster Label'] == 0]

,Borough,Area,Population,Latitude,Max_Rent,Longitude,Cluster Label,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
1,Bexley,23.38,248287,39.969238,97.0,-82.936864,0,Coffee Shop,Ice Cream Shop,Pizza Place,Discount Store,Chinese Restaurant
5,Havering,43.35,259552,51.544385,86.0,-0.144307,0,Pub,Hotel,Coffee Shop,Park,Bakery


Havering and Bexley are good fits to our target boroughs: considering entertainment venues, big area and quite population. Havering has a slight financial edge - cheaper rent a more incline to pubs and park in the area.

In [23]:
Borough_merged.loc[Borough_merged['Cluster Label'] == 1]

,Borough,Area,Population,Latitude,Max_Rent,Longitude,Cluster Label,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
4,Haringey,11.42,268647,51.58793,107.75,-0.105410,1,Pub,Café,Coffee Shop,Park,Turkish Restaurant
6,Merton,14.52,206548,51.41087,123.75,-0.188097,1,Pub,Park,Bar,Coffee Shop,Sushi Restaurant
7,Redbridge,21.78,305222,51.57632,118.50,0.045410,1,Pub,Park,Coffee Shop,Restaurant,Café


Pub as a most popular venue, and for those, who dont feel like a visit to a pub can enjoy the nature.
This cluster is hardly of interest, because it will be problematic to open a new Pub. Not to mention a Park...

In [24]:
Borough_merged.loc[Borough_merged['Cluster Label'] == 2]

,Borough,Area,Population,Latitude,Max_Rent,Longitude,Cluster Label,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,Barking and Dagenham,13.93,212906,51.554117,102.25,0.150504,2,Grocery Store,Park,Supermarket,Pub,Coffee Shop
2,Bromley,57.97,332336,51.402805,118.50,0.014814,2,Pub,Coffee Shop,Grocery Store,Park,Pizza Place
3,Enfield,31.74,333794,51.652085,102.25,-0.081018,2,Coffee Shop,Park,Pub,Turkish Restaurant,Café


Even though the Pub is a top venue, it is narrowly followed by Coffee Shop, Park and Turkish Restaurant. I would not call visit to the grocery store as a social route. With a sizeable area and population of the borough, Bromley and Enfield if a good choice for a new venue of a similar sort.

## 8. Observations

Results of the above analysis and clustering can be summarized:
1. The most popular social venues, ouside of Inner London boroughs are Pubs and Coffee shops
2. Northern boroughs are more prone to visiting pubs, whereas southern boroughs are most likely to shop and have the social life from home
3. Within top 5 places of interest in every borough is an ethnic restaurant
4. Rent price is not so much a factor for going out - the demand is not affected by difference in costs

## 9. Discussion

Looking at the data, Havering, Bromley and Enfield are the best places outside of Central London where a new venue is worth opening. However, a lot of information is not taken into account, and cannot be obtained from Foursquare Developer:
1. Higher ethnic presence in a given borough can and will influence the popularity of a given cuisine.
2. Closer proximity to Inner boroughs and better transport links allows people to travel to the neighbouring borough and impact the measurements
3. Many small venues are not registered in Foursquare and are marketed via word-of-mouth, and are not taken into account

Regardless, the analysis provided an insight into what people like and opt for, when it comes to going out in their own neighbourhoods.


## 10. Conclusion

Finally to conclude this project, I have had a good trial run at solving a real-life problem, using available data to find a business solution - choosing to open a venue in London .I have made use of some frequently used python librairies to manipilate data, use Foursquare API to explore the information on the Boroughs I looked into and managed to make a map of results, that allowed me to illustrate my point graphically and quite clearly to someone, not familiar with data manipulation and who only wants to know one thing - where will my venue be flourishing??